In [1]:
import os
print("Current directory:", os.getcwd())
print("\nFiles in directory:")
for file in os.listdir():
    print(f"  - {file}")

Current directory: C:\Users\caleb\anaconda_projects\satellite-los-analysis

Files in directory:
  - .gitignore.txt
  - .ipynb_checkpoints
  - constellation_TLE.ipynb
  - data
  - docs
  - fetch_globalstar.py
  - Globalstar_LOS_Analysis.ipynb
  - gp.php
  - LICENSE.txt
  - maps
  - output
  - README.txt
  - requirements.txt
  - satellite_los_analysis-constell.ipynb
  - satellite_los_analysis.ipynb
  - satellite_los_analysis.ipynb.txt
  - __pycache__


In [2]:
# Cell 1: Test module import
print("Testing module import...")

try:
    from fetch_globalstar import fetch_globalstar_satellites, print_satellite_summary
    print("✓ Module imported successfully!")
except ImportError as e:
    print(f"✗ Import failed: {e}")
    print("\nTroubleshooting:")
    print("1. Check that fetch_globalstar.py is in the same directory as this notebook")
    print("2. Verify the filename is exactly 'fetch_globalstar.py'")
    print("3. Restart the kernel: Kernel → Restart")

Testing module import...
✓ Module imported successfully!


In [3]:
# Cell 2: Initialize Skyfield timescale
from skyfield.api import load

print("Initializing Skyfield timescale...")
ts = load.timescale()
print("✓ Timescale initialized")

Initializing Skyfield timescale...
✓ Timescale initialized


In [4]:
# Cell 3: Fetch Globalstar satellite TLE data
print("="*60)
print("FETCHING GLOBALSTAR SATELLITES")
print("="*60)

# This will download fresh TLE data from CelesTrak
globalstar_fleet = fetch_globalstar_satellites(ts)

# Display summary
print_satellite_summary(globalstar_fleet)

# Store count
NUM_SATELLITES = len(globalstar_fleet)
print(f"✓ Ready to analyze {NUM_SATELLITES} satellites")

FETCHING GLOBALSTAR SATELLITES
Fetching Globalstar TLE data from CelesTrak...
✓ Downloaded 14280 bytes
Filtering for satellites M073 through M097...
✓ Loaded 25 satellites

GLOBALSTAR SATELLITE FLEET (25 satellites)
 1. GLOBALSTAR M079
 2. GLOBALSTAR M074
 3. GLOBALSTAR M076
 4. GLOBALSTAR M077
 5. GLOBALSTAR M075
 6. GLOBALSTAR M073
 7. GLOBALSTAR M083
 8. GLOBALSTAR M088
 9. GLOBALSTAR M091
10. GLOBALSTAR M085
11. GLOBALSTAR M081
12. GLOBALSTAR M089
13. GLOBALSTAR M084
14. GLOBALSTAR M080
15. GLOBALSTAR M082
16. GLOBALSTAR M092
17. GLOBALSTAR M090
18. GLOBALSTAR M086
19. GLOBALSTAR M097
20. GLOBALSTAR M093
21. GLOBALSTAR M094
22. GLOBALSTAR M096
23. GLOBALSTAR M078
24. GLOBALSTAR M095
25. GLOBALSTAR M087

✓ Ready to analyze 25 satellites


In [6]:
# Cell 4: Verify satellite data loaded correctly
print("VERIFICATION CHECKS")
print("="*60)

# Check 1: Correct number of satellites
print(f"✓ Loaded {len(globalstar_fleet)} satellites")

# Check 2: First satellite details
first_sat = globalstar_fleet[0]
print(f"✓ First satellite: {first_sat.name}")
print(f"  Type: {type(first_sat)}")

# Check 3: Last satellite details
last_sat = globalstar_fleet[-1]
print(f"✓ Last satellite: {last_sat.name}")

# Check 4: All satellite names
print(f"\nAll satellite names:")
for sat in globalstar_fleet:
    print(f"  - {sat.name}")

VERIFICATION CHECKS
✓ Loaded 25 satellites
✓ First satellite: GLOBALSTAR M079
  Type: <class 'skyfield.sgp4lib.EarthSatellite'>
✓ Last satellite: GLOBALSTAR M087

All satellite names:
  - GLOBALSTAR M079
  - GLOBALSTAR M074
  - GLOBALSTAR M076
  - GLOBALSTAR M077
  - GLOBALSTAR M075
  - GLOBALSTAR M073
  - GLOBALSTAR M083
  - GLOBALSTAR M088
  - GLOBALSTAR M091
  - GLOBALSTAR M085
  - GLOBALSTAR M081
  - GLOBALSTAR M089
  - GLOBALSTAR M084
  - GLOBALSTAR M080
  - GLOBALSTAR M082
  - GLOBALSTAR M092
  - GLOBALSTAR M090
  - GLOBALSTAR M086
  - GLOBALSTAR M097
  - GLOBALSTAR M093
  - GLOBALSTAR M094
  - GLOBALSTAR M096
  - GLOBALSTAR M078
  - GLOBALSTAR M095
  - GLOBALSTAR M087


In [5]:
# Cell 5: Calculate satellite positions at current time
from skyfield.api import wgs84
from datetime import datetime, timezone

# Define observer location (Woods Canyon example)
OBSERVER_LAT = 33.52    # degrees North
OBSERVER_LON = -117.75  # degrees West
OBSERVER_ALT = 100      # meters

observer = wgs84.latlon(OBSERVER_LAT, OBSERVER_LON, OBSERVER_ALT)

# Get current time (timezone-aware)
t = ts.now()
current_time = datetime.now(timezone.utc)  # ✓ Fixed: Use timezone-aware datetime

print(f"Observer: {OBSERVER_LAT}°N, {OBSERVER_LON}°W, {OBSERVER_ALT}m")
print(f"Time: {current_time.strftime('%Y-%m-%d %H:%M:%S')} UTC")
print("="*70)
print(f"{'Satellite':<20} {'Elevation':>10} {'Azimuth':>10} {'Distance':>12} {'Status':>8}")
print("="*70)

visible_count = 0

for sat in globalstar_fleet:
    difference = sat - observer
    topocentric = difference.at(t)
    alt, az, distance = topocentric.altaz()
    
    is_visible = alt.degrees > 0
    if is_visible:
        visible_count += 1
    
    status = "VISIBLE" if is_visible else "Below"
    print(f"{sat.name:<20} {alt.degrees:>9.2f}° {az.degrees:>9.2f}° {distance.km:>10.1f} km {status:>8}")

print("="*70)
print(f"\nSummary: {visible_count}/{NUM_SATELLITES} satellites visible above horizon")

Observer: 33.52°N, -117.75°W, 100m
Time: 2025-11-17 22:36:53 UTC
Satellite             Elevation    Azimuth     Distance   Status
GLOBALSTAR M079         -50.32°     60.00°    11558.7 km    Below
GLOBALSTAR M074         -35.31°    179.84°     9466.9 km    Below
GLOBALSTAR M076         -32.36°    303.81°     9053.4 km    Below
GLOBALSTAR M077         -33.04°     82.23°     9537.1 km    Below
GLOBALSTAR M075         -38.41°    334.27°     9953.6 km    Below
GLOBALSTAR M073         -37.80°     28.02°     9865.7 km    Below
GLOBALSTAR M083           2.79°    123.71°     4172.8 km  VISIBLE
GLOBALSTAR M088         -18.84°     67.81°     6994.8 km    Below
GLOBALSTAR M091          -3.02°    309.44°     4833.8 km    Below
GLOBALSTAR M085          38.70°    275.33°     2008.6 km  VISIBLE
GLOBALSTAR M081         -60.24°    323.10°    12665.6 km    Below
GLOBALSTAR M089         -11.88°    200.14°     5965.5 km    Below
GLOBALSTAR M084         -52.23°      8.60°    11797.0 km    Below
GLOBALSTAR M

In [6]:
# Cell 6: Integration with existing LOS analysis
print("Ready to integrate with terrain-based line-of-sight analysis")
print(f"Variables available:")
print(f"  - globalstar_fleet: {len(globalstar_fleet)} satellites")
print(f"  - observer: Observer location object")
print(f"  - t: Current time")
print(f"\nNext step: Run your existing DEM-based LOS analysis")
print(f"for each satellite in globalstar_fleet")

Ready to integrate with terrain-based line-of-sight analysis
Variables available:
  - globalstar_fleet: 25 satellites
  - observer: Observer location object
  - t: Current time

Next step: Run your existing DEM-based LOS analysis
for each satellite in globalstar_fleet
